### Suitability Analysis

Este cuaderno tiene como objetivo realizar un análisis de aptitud espacial en el departamento del Valle del Cauca. A partir de los datos previamente preparados y reproyectados en el sistema de referencia oficial MAGNA-SIRGAS (EPSG:3116), se integrarán diversas capas temáticas para identificar las zonas más adecuadas para conservación ecológica.

## Capas utilizadas:
- Vías
- Áreas protegidas
- Curvas de nivel
- Pendientes
- Cobertura del suelo
- Hidrografía

## Metodología general:
1. **Normalización de criterios**: se asignarán valores a cada capa según su influencia positiva o negativa en la aptitud.
2. **Combinación ponderada**: se aplicarán pesos a cada capa según su importancia relativa.
3. **Análisis espacial**: se generará un mapa de aptitud mediante álgebra de mapas.
4. **Clasificación**: el resultado se clasificará en zonas de alta, media y baja aptitud.


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

## Cargar capas reproyectadas y recortadas al Valle del Cauca

gdf_cartografia = gpd.read_file(cartografia)
gdf_vias = gpd.read_file(cartografia)
gdf_protegidas = gpd.read_file(proteccion)
gdf_curvas = gpd.read_file(curvas)
gdf_pendiente = gpd.read_file(pendiente)
gdf_suelo = gpd.read_file(suelo)
gdf_hidrografia = gpd.read_file(hidrografia)

In [ ]:
from shapely.geometry import Point
from shapely.ops import nearest_points
from scipy.spatial import cKDTree

# 1. Crear una grilla de puntos dentro del Valle del Cauca
from rasterio.features import geometry_mask
from rasterio.transform import from_origin

# Convertir a un solo polígono
valle_union = vias_valle.unary_union

# Ajustar resolución según el tamaño del área
xmin, ymin, xmax, ymax = valle_union.bounds

res = 1000  # tamaño de celda en metros
x_coords = np.arange(xmin, xmax, res)
y_coords = np.arange(ymin, ymax, res)
grid_points = [Point(x, y) for x in x_coords for y in y_coords]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs=vias_valle.crs)

# 2. Calcular distancia mínima de cada punto a la vía más cercana
vias_union = vias_valle.unary_union  # unifica todos los segmentos de vía
grid_gdf["dist_vias"] = grid_gdf.geometry.apply(lambda point: point.distance(vias_union))

# 3. Visualizar mapa de distancias
fig, ax = plt.subplots(figsize=(10, 8))
grid_gdf.plot(column="dist_vias", cmap="YlOrRd", legend=True, ax=ax)
vias_valle.plot(ax=ax, color='black', linewidth=0.5)
plt.title("Distancia a Vías (metros)")
plt.show()


In [1]:
#Prueba